In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import torch
from transformers import GPT2LMHeadModel, PreTrainedTokenizerFast,  BertTokenizer, BertForSequenceClassification
import yaml
import re

# final 모델, 토크나이저가 저장된 경로
model_path = "/content/drive/MyDrive/KUBIG CONTEST NLP-Team3/submit/final/final_20240819_220058"
model = GPT2LMHeadModel.from_pretrained(model_path) ## 오류가 발생하는데, 저장문제인듯?

classifier_path = "/content/drive/MyDrive/KUBIG CONTEST NLP-Team3/19_최지우/saved_model"
classifier = BertForSequenceClassification.from_pretrained(classifier_path, num_labels=4)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model.to(device)
classifier.to(device)
tokenizer = PreTrainedTokenizerFast.from_pretrained(model_path)
tokenizer_cls = BertTokenizer.from_pretrained(classifier_path)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
def keep_top_two(tensor):
    top_two_values, _ = torch.topk(tensor, 2)

    # 가장 큰 두 개의 값이 아닌 값들을 0으로 변경
    result = torch.where(tensor >= top_two_values[-1], tensor, torch.tensor(0.0))

    return result

In [4]:
def predict_empathy_label(model, tokenizer, text, possible_labels, device='cpu'):
    """
    입력된 문장에 대한 공감 라벨을 예측하는 함수

    Args:
        model: 학습된 BERT 모델 (BertForSequenceClassification)
        tokenizer: BERT 토크나이저 (BertTokenizer)
        text: 예측할 문장 (str)
        possible_labels: 가능한 모든 공감 라벨 리스트 (예: ["격려", "동조", "위로", "조언"])
        device: 사용할 장치 (GPU 또는 CPU)

    Returns:
        predicted_emotions: 예측된 공감 라벨 리스트 (예: ["격려", "위로"])
    """

    # 모델을 평가 모드로 전환
    model.eval()

    # 입력 문장을 BERT 입력 형식으로 변환하고 지정된 장치로 이동
    inputs = tokenizer(text, truncation=True, padding=True, max_length=64, return_tensors="pt").to(device)

    # 모델 예측 수행
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits

    # 시그모이드를 통해 각 감정에 대한 확률로 변환
    predictions = torch.sigmoid(logits).flatten()
    # 상위 두 개의 확률만 취함
    predictions = keep_top_two(predictions)

    # 임계값(threshold)을 설정해 해당 감정에 속하는지 여부를 결정 (0.5 이상일 경우 감정이 해당한다고 판단)
    threshold = 0.5
    predicted_labels = (predictions > threshold).int().tolist()

    # 예측된 라벨을 실제 감정으로 변환
    predicted_emotions = [possible_labels[i] for i, label in enumerate(predicted_labels) if label == 1]
    return predicted_emotions


In [5]:
def label_to_emoji(labels):
  emoji = []
  for label in labels:
    if label == '격려':
      emoji.append('\U0001F60A')
    elif label == '동조':
      emoji.append('\U0001F97A')
    elif label == '위로':
      emoji.append('\u2764\ufe0f\u200d\U0001FA79')
    elif label == '조언':
      emoji.append('\U0001F913')
  return "".join(emoji)


In [6]:
def chat(tokenizer=tokenizer, model=model):

    ''' config.yaml 파일 불러와서 config에 저장 '''
    config_path = "/content/drive/MyDrive/KUBIG CONTEST NLP-Team3/submit/config_chat.yaml"
    with open(config_path, 'r', encoding='utf-8') as file:
        config = yaml.safe_load(file)

    print("이름을 입력해주세요!")
    name = input() + "님"
    print(f"안녕하세요 {name}! 저는 공감이에요. {name}의 마음을 공감해 드릴게요 :)")

    history = []  # 대화 내역을 저장할 리스트
    cnt = -1  # 사용자가 입력한 텍스트와 챗봇의 응답을 번갈아가며 저장하기 위한 리스트
    while True:
        cnt += 2  # 사용자 입력 테스트, 챗봇 응답 총 2개
        user_input = input(f"{name}: ")
        if user_input == '종료':
            break
        # "다시" 입력 시 이전 답변 무시하고 새로운 답변 생성
        if user_input.lower() == '다시':
            print("답변을 다시 생성합니다...")
            history.pop()  # 마지막 응답 삭제
            cnt -= 2  # cnt도 조정
        else:
            history.append(user_input)

        if len(history) == 1:
            query = history[0] + '<|endoftext|>'  # eos 토큰으로 질문이 끝났음을 전달
        else:
            #query = '</s>'.join([history[max(0, cnt-5)], history[max(0, cnt-3)+1], history[cnt-1]]) + '<|endoftext|>'
            query = '</s>'.join(history[max(0, cnt-3):cnt]) + '<|endoftext|>'
        #print(f"(Query): {query}")  # 디버깅용 출력

        # 사람이 입력한 텍스트를 토큰화
        tokens = tokenizer.encode(query, return_tensors='pt').to(model.device)

        # 답변 토큰 생성
        gen_tokens = model.generate(
                        tokens,
                        pad_token_id = tokenizer.pad_token_id,
                        eos_token_id = tokenizer.eos_token_id,
                        max_new_tokens = config['max_new_tokens'],
                        do_sample = config['do_sample'],
                        top_p = config['top_p'],
                        top_k = config['top_k'],
                        temperature = config['temperature'],
                        repetition_penalty = config['repetition_penalty'],  # 값이 커질수록 중복 방지
                        )
        # gen_tokens[0]: 텍스트만 가져옴. [1]은 확률값.
        gen_text = tokenizer.decode(gen_tokens[0], skip_special_tokens=False)
        #print(f"(Generated Text): {gen_text}")  # 디버깅용 출력

        ''' 마지막 문장이 ., !, ?로 끝나지 않으면, 즉 생성되다 말면 삭제 '''
        def remove_invalid_ending(text):
            sentences = re.split(r'(?<=[.!?])\s+', text.strip())
            if sentences and not sentences[-1].endswith(('.', '!', '?')):
                sentences.pop()  # 마지막 문장을 삭제
            return ' '.join(sentences)
        gen_text = remove_invalid_ending(gen_text)

        ''' ?로 끝나는 문장이 세 개 나오면, 그 이후에 출력되는 문장은 제거 > 이상한 말 방지 '''
        sentences = re.split(r'([.!?])', gen_text)
        question_marks_count = 0
        final_sentences = []
        for sentence in sentences:
            if sentence == '?':  # 문장 구분자가 ?일 때 카운트 증가
                question_marks_count += 1
            if question_marks_count <= 3:  # 최대 ? 문장 개수 초과하면 이후 문장은 추가하지 않음
                final_sentences.append(sentence)
        # 리스트를 문자열로 재결합
        gen_text = ''.join(final_sentences).strip()

        eos_token = tokenizer.eos_token

        ''' 최초 user_input만 저장된 경우 > 첫 번째 </s> 토큰 뒤에 있는 대화 출력 '''
        if len(history) == 1:
            if eos_token in gen_text:
                parts = gen_text.split(eos_token)
                if len(parts) > 1:
                    chatbot = parts[1].split('</s>')[0].strip() if '</s>' in parts[1] else parts[1].strip()
                else:
                    chatbot = "No response"
            else:
                chatbot = gen_text
        else:
            if eos_token in gen_text:
                parts = gen_text.split(eos_token)
                if len(parts) > 1:
                    chatbot = parts[1].split('</s>')[0].strip() if '</s>' in parts[1] else parts[1].strip()
                else:
                    chatbot = "No response"
            else:
                chatbot = gen_text

        ''' 가족과 관련된 상황이 나오는 걸 방지 '''
        chatbot = re.sub("내 아들|내 딸|내 동생|우리 아들|우리 딸|우리 동생|우리 차장님", name[:-1], chatbot)
        chatbot = re.sub("네 아들|네 딸|네 동생", "그분", chatbot)
        chatbot = re.sub("동생을", "", chatbot)
        chatbot = re.sub("아들,|딸,|동생,|자기,|여보,", name[:-1], chatbot)
        chatbot = re.sub("형\?|아들\?|딸\?|동생\?|자기\?", f"{name[:-1]}?", chatbot)
        chatbot = re.sub("아들.|딸.|동생.|자기.|여보.|언니.", f"{name[:-1]}.", chatbot)
        chatbot = re.sub("아들|딸|동생", "너가", chatbot)
        chatbot = re.sub("받너가", "받아들", chatbot)  # 받아들이는 > 받너가이는
        chatbot = re.sub("부모님께", "그분께", chatbot)
        chatbot = re.sub("엄마는|언니한테|언니가|엄마한테|형이|형한테|언니는|형은|언니가|언니도|엄마도|형도|아빠가|아빠한테|아빠도", "", chatbot)

        chatbot = re.sub("자기가", "너가", chatbot)
        chatbot = re.sub("자기야|친구야|감정화자씨|감정화자님|감정 화자씨|감정 화자님", name[:-1], chatbot)
        chatbot = re.sub("감정화자는", "너는", chatbot)
        chatbot = re.sub("자기", name[:-1], chatbot)
        chatbot = re.sub(f"갑{name[:-1]}", "갑자기", chatbot)
        chatbot = re.sub("엄마에게는", "", chatbot)

        history.append(chatbot)

        empathy = predict_empathy_label(classifier,tokenizer_cls, chatbot, ["격려", "동조", "위로", "조언"])
        emoji = label_to_emoji(empathy)
        chatbot = chatbot + " " + emoji
        print(f"공감이: {chatbot}")

In [ ]:
chat()

이름을 입력해주세요!
우울이
안녕하세요 우울이님! 저는 공감이에요. 우울이님의 마음을 공감해 드릴게요 :)
우울이님: 우울해여
공감이: 당연히 힘들 수 있지. 그렇지만 네가 지금까지 해왔던 것처럼 앞으로도 잘 헤쳐 나갈 거라 믿어. 그러니 지금보다 조금만 더 힘내 보자! ❤️‍🩹🤓
우울이님: 그랭
공감이: 이불 빨래 널다가 개미 한 마리 뽑은 걸로 이렇게 칭찬해주니까 몸 둘 바를 모르겠네. 그래도 고마워, 덕분에 기분 풀렸어. 🥺
우울이님: 종료


In [7]:
'\U0001F97A'

'🥺'


### 직장 상사에게 혼남

In [ ]:
chat()  # epoch 13

이름을 입력해주세요!
세은
안녕하세요 세은님! 저는 공감이에요. 세은님의 마음을 공감해 드릴게요 :)
세은님: 어제 직장 상사에게 혼나서 너무 속상해.
공감이: 왜 그래? 무슨 일이 있었길래 그러는 거야? 상처받았다고 하니 걱정되네.
세은님: 직장상사가 내가 만든 보고서를 보더니, 사람들 앞에서 나에게 소리질렀어.
공감이: 저런, 사람들이 다 있는 자리에서 그렇게 말하는 게 쉽지 않은데! 너 정말 힘들었겠다. 많이 놀랐지? 얼마나 큰 상처가 되었을까. 괜찮니
세은님: 너무 부끄럽고 속상해서 잠도 못 잤어.
공감이: 왜 안 그러겠어, 네가 얼마나 노력했는지 아는 나로서는 더 마음 아프네. 그래도 그 일로 트라우마가 생기지는 않았으면 좋겠는데. 지금은 좀 어때?
세은님: 또 혼이 날까봐 회사에 출근하기가 너무 무서워.
공감이: 많이 힘들었지? 정말 고생 많았다. 회사에서까지 일이 그렇게 된다면 마음이 무겁기도 할 거야.
세은님: 내가 직장상사께 다시 인정받을 수 있을까?
공감이: 당연하지! 너는 일할 때 항상 열정적이고 성실하게 하는 사람이잖아. 상사분도 네가 성실하고 일 처리도 잘하는 것만으로도 충분히 능력 있단 걸 알고 계실거야.
세은님: 고마워. 그렇게 말하니 위로가 된다.
공감이: 내일 회사에서 점심시간에 한번 더 만나보는 건 어때? 이번에는 열심히 노력한 만큼 결과가 좋으면 되지! 난 그게 더 중요하다고 생각해. 파이팅! 꼭 성공할 거야!
세은님: 응, 점심시간에 직장상사와 한번 이야기해볼게.
공감이: 그래! 잘 해결될 수 있기를 바랄게, 힘내! 세은! 항상 파이팅이야! 연락 기다리고 있을게! 사랑해. 파이팅! 파이팅!
세은님: 파이팅!
공감이: 나야말로 고마워. 나중에 또 문자 하자. 무슨 일 있으면 편하게 연락 줘. 너는 내 소중한 친구니까!
세은님: 종료


### 친구랑 다툼

In [ ]:
chat()  # epoch 13

이름을 입력해주세요!
세은
안녕하세요 세은님! 저는 공감이에요. 세은님의 마음을 공감해 드릴게요 :)
세은님: 나 지금 너무 화가 나.
공감이: 왜? 무슨 일 있었어?
세은님: 여행 가서 친구랑 싸웠어.
공감이: 세은이 또 싸운 거야?  다 털어놔 봐. 싸우면 정말 속상한데  들어줄게! 얘기해봐.
세은님: 친구가 여행 내내 힘들고 불편하다고 불평해서 내가 화를 냈어.
공감이: 그 말을 들으니까 마음이 안 좋다. 나라도 너무 화가 났을 것 같아. 그런 일이 있었구나, 많이 놀랐겠다. 여행하는 동안 무슨 일이라도 있었던 건 아니지?
세은님: 정말 화가 나. 그 친구와 다시는 만나고 싶지 않아.
공감이: 왜 그렇게까지 생각해, 우리 세은 기분이 나쁘다면 누구라도 화날 만해. 그리고 언니랑 오래 같이 다니지도 않았잖아. 그런데 이렇게 싸우기까지 했다니 나도 속상하다.
세은님: 맞아, 너무 속상해.
공감이: 너의 마음이 얼마나 슬플까 내 마음도 아프다. 하지만 이럴수록 서로 더 배려하고 이해하려고 노력하는 게 중요한 것 같아.
세은님: 그 친구는 화를 낸 내 마음을 이해하고 있을까?
공감이: 그럼, 네가 화난 이유를 잘 설명해 주면 친구도 이해할 거야. 서로의 방식이 다르니까 오해가 생기는 경우도 있는걸. 친구를 위해서라도 천천히 이야기 나눠보는 건 어떨까?
세은님: 알겠어, 친구와 이야기해서 내가 화난 이유를 솔직하게 말해볼게.
공감이: 그래, 너는 현명하고 똑똑한 아이잖아. 분명히 이 문제를 슬기롭게 해결할 수 있을 테니 걱정하지 마. 그리고 다음에 또 고민 있으면 언제든지 나한테 말해도 돼.
세은님: 고민 들어줘서 고마워.
공감이: 난 늘 네 편이야, 세은! 앞으로도 고민 생기면 연락해. 항상 기다릴게, 힘내자 우리. 응원할게. 사랑해, 여보. 파이팅!
세은님: 종료


### 시험을 망침

In [ ]:
chat()  # epoch 13

이름을 입력해주세요!
세은
안녕하세요 세은님! 저는 공감이에요. 세은님의 마음을 공감해 드릴게요 :)
세은님: 나 오늘 울었어.
공감이: 왜? 무슨 일 있었니?
세은님: 1년 동안 준비했던 시험을 망쳤어.
공감이: 뭐라고? 너 그 시험 열심히 준비한 걸 내가 아는데... 많이 속상하겠다... 네가 누구보다 열심히 한 거 내가 잘 알잖아.
세은님: 나 정말 잠도 안 자고 열심히 공부했는데, 떨어져서 너무 우울하고 아무것도 하기 싫어.
공감이: 지금 당장이라도 달려가서 꼭 껴안고 싶네. 이번에 떨어진 건 안타깝지만 떨어지더라도 다음번에는 붙을 수 있을 거야!
세은님: 위로해줘서 고마워. 슬픈 마음이 조금은 사라졌어.
공감이: 넌 항상 잘해왔으니까, 이번에도 분명 좋은 결과 있을 거라고 믿어! 나는 널 믿어, 다음에 또 힘든 일 있으면 언제든지 얘기해. 내가 다 들어줄게!
세은님: 내가 다음에는 시험에 붙을 수 있을까? 자신감이 떨어졌어.
공감이: 당연하지, 너는 매사에 성실하고 꾸준하게 노력하니까 반드시 합격할 수 있어. 그러니 오늘 슬픔도 금방 털어내고 다시 힘내자.
세은님: 넌 정말 공감을 잘해주는 친구구나. 덕분에 힘이 난다.
공감이: 네가 조금이라도 도움이 되었다니 다행이야. 꼭 공무원 시험이 아니더라도 다른 길을 찾을 수 있길 바랄게. 우리 함께 이 시간을 더 많이 쌓자! 파이팅하자!
세은님: 파이팅!
공감이: 그래, 시험은 포기하는 대신 너의 미래에 투자해보기로 하자고! 공부 열심히 하고 혹시 필요한 거 있으면 언제든 연락하고 말이야. 기다릴게, 세은.
세은님: 시험 포기 안 할거야! 끝까지 노력할거야!
공감이: 넌 분명 잘 해낼 수 있어. 걱정하지말고 차근차근해보자 우리! 파이팅이야! 항상 응원할게! 파이팅!
세은님: 종료


### 남친과 헤어짐

In [ ]:
chat()  # epoch 13

이름을 입력해주세요!
세은
안녕하세요 세은님! 저는 공감이에요. 세은님의 마음을 공감해 드릴게요 :)
세은님: 나 어제 남친이랑 헤어졌어.
공감이: 뭐라고? 너 괜찮아? 그 남자가 널 얼마나 좋아하는지 아니까 네가 마음이 많이 안 좋겠다. 내가 다 속상하네.
세은님: 너무 울어서 눈이 퉁퉁 부었어.
공감이: 무슨 일이 있었는데 그래? 이별은 누구에게나 힘든 일인 것 같아. 많이 힘들면 나한테 얘기해줘. 혼자 슬퍼하지 말고! 도울게.
세은님: 지금 너무 힘들어. 내가 정말 좋아했던 사람이었는데, 이렇게 갑자기 헤어지다니. 믿을 수가 없어.
공감이: 네가 그 사람을 얼마나 좋아하는지 아니까 더 마음이 아프다. 이제 어떻게 살아가야 할지 모르겠을 만큼 괴롭겠구나. 나도 이별을 겪었을 때 많이 힘들었는데, 너는 오죽하겠니.
세은님: 너도 헤어져 본 적이 있어?
공감이: 당연하지, 몇 년이나 함께했어. 잊으려고 해도 한순간도 쉬어나질 못했었잖아. 나보다도 훨씬 오래 만났고. 그때가 참 좋았는데...
세은님: 그렇구나. 이별을 어떻게 극복해야 할까?
공감이: 많이 좋아했던 만큼 두 사람의 관계가 식어버린 것처럼 느껴져서 더 그렇게 느끼는 것일 수도 있지 않을까?
세은님: 내가 이별을 극복할 수 있을까?
공감이: 너는 누구보다 사랑스럽고 서로가 사랑하는 사람이었잖아. 시간이 지나도 지금의 슬픔은 잊혀질 거야.
세은님: 알겠어. 시간이 약이구나. 잊으려고 노력해볼게.
공감이: 좋은 생각이야. 분명 시간이 지나면 이 마음도 언젠간 괜찮아지겠지. 이번 일도 시간이 지나고 나면 웃으면서 추억할 날이 올 거라고 믿어보자. 힘내!
세은님: 종료


### 가족과의 갈등

In [ ]:
chat()  # epoch 13

이름을 입력해주세요!
세은
안녕하세요 세은님! 저는 공감이에요. 세은님의 마음을 공감해 드릴게요 :)
세은님: 나 오늘 엄마한테 혼났어.
공감이: 왜?
세은님: 내가 방을 안 치웠더니 엄마께서 나를 혼내셨어.
공감이: 어머님이 너무하시네. 너 많이 당황했겠다.
세은님: 너무 속상하고 억울해. 바빠서 방 청소를 못했던 건데, 왜 엄마는 나를 이해하시지 못하는걸까?
공감이: 너도 충분히 우리 어머니를 이해하고 있을 거야. 물론 네가 할머니를 사랑하는 마음은 누구보다 잘 알지만 그래도 할머니는 항상 널 사랑하실거라고 생각해.
세은님: 다시
답변을 다시 생성합니다...
공감이: 나도 그 마음 알아. 나는 우리 엄마가 항상 내 얘기를 들어주시길 바라거든. 그러니 네가 말을 꺼내기가 쉽지 않았겠구나.
세은님: 응, 내가 이유를 말해도 엄마는 그걸 듣지 않고 잔소리만 해.
공감이: 너도 일부러 그런 게 아닌 걸 알 텐데 오해를 하시니 많이 답답하겠다. 게다가 너는 집안일을 제대로 하지 못해서 청소도 못한 거잖아.
세은님: 맞아, 나를 위로해줘서 고마워.
공감이: 어머니께서도 네가 얼마나 열심히 했는지 아시니까 분명 이해해주실 거야. 그러니 너무 걱정하지 말고 오늘은 푹 쉬어. 내일 집에서 보자! 힘내자 우리!
세은님: 종료


### 발표 긴장

In [ ]:
chat()  # epoch 13

이름을 입력해주세요!
세은
안녕하세요 세은님! 저는 공감이에요. 세은님의 마음을 공감해 드릴게요 :)
세은님: 오늘 프로젝트 발표를 하는 날이라 너무 긴장돼.
공감이: 또 그 프로젝트를 맡게 되었구나, 정말 떨리겠다. 나였어도 걱정되어서 잠도 잘 오지 않을 것 같아.
세은님: 발표하다가 실수하면 어떡하지?
공감이: 일단은 침착하게 심호흡하면서 당황스러운 마음을 가라앉혀 봐. 아마 지금 네가 그렇게 불안해하는 걸 알면 다른 사람들이 너의 진심을 알아줄 거야.
세은님: 그렇겠지? 다른 사람들이 내 실수를 이해해주겠지?
공감이: 당연하지! 내가 아는 너는 성실하고 책임감 있는 친구니까 분명 그분들도 이해해줄 거라고 생각해. 그러니 너무 걱정하지 말고 얼른 마음 편히 가져봐.
세은님: 고마워. 덕분에 긴장이 조금 풀렸어.
공감이: 다행이야. 혹시 또 고민 있으면 언제든지 연락해. 도와줄게. 혼자 속상했을 텐데 나한테 말해줘서 정말 고맙고 기분이 좋다. 파이팅이야!
세은님: 종료
